## Imports


In [ ]:
import torch
import torch.nn as nn 
import torch.nn.functional as F 
from torch import device
from torch.optim.lr_scheduler import LambdaLR
from dataclasses import dataclass

import math
import tiktoken
import os

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("Using device:", device)
torch.set_float32_matmul_precision('high')

## Model Architecture

In [ ]:
@dataclass
class gpt2config:
    n_vocab: int = 50257
    n_layer: int = 12
    n_embed: int = 128
    n_context: int = 1024
    n_head: int = 8
    n_timesteps: int = 1000
    mlp_expansion: int = 4
    n_latent: int = 768


In [ ]:
class GPT2Attention(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.c_attn = nn.Linear(config.n_latent, 3 * config.n_latent)
        self.c_proj = nn.Linear(config.n_latent, config.n_latent)
        self.n_head = config.n_head
        self.n_embed = config.n_embed
        self.n_latent = config.n_latent
        # Create a causal mask (lower triangular matrix) and register it as a buffer
        # A buffer is not a parameter, but is saved with the model state_dict
        self.register_buffer("bias", torch.tril(torch.ones(config.n_context, config.n_context))
                                     .view(1, 1, config.n_context, config.n_context))

    def forward(self, x):
        B, T, C = x.size()
        
        # Calculate query, key, values for all heads in batch
        qkv = self.c_attn(x)
        q, k, v = qkv.split(self.n_latent, dim=2)
        
        # Reshape for multi-head attention: (B, nh, T, hs)
        k = k.view(B, T, self.n_head, C // self.n_head).transpose(1, 2)
        q = q.view(B, T, self.n_head, C // self.n_head).transpose(1, 2)
        v = v.view(B, T, self.n_head, C // self.n_head).transpose(1, 2)

        # Scaled dot-product attention
        att = (q @ k.transpose(-2, -1)) * (1.0 / (k.size(-1) ** 0.5))
        
        # --- MASKING STARTS HERE ---
        # Apply the causal mask: fill "future" positions with -infinity
        # This makes their softmax probability zero.
        # att = att.masked_fill(self.bias[:, :, :T, :T] == 0, float('-inf'))
        # --- MASKING ENDS HERE ---

        att = F.softmax(att, dim=-1)
        y = att @ v # (B, nh, T, hs)
        
        # Re-assemble all head outputs side-by-side
        y = y.transpose(1, 2).contiguous().view(B, T, C)
        
        # Output projection
        y = self.c_proj(y)
        return y
    
class GPT2MLP(nn.Module):
    def __init__(self,config):
        super().__init__()
        
        self.c_fc = nn.Linear(config.n_latent, config.mlp_expansion*config.n_latent)
        self.act = nn.GELU(approximate="tanh")
        self.c_proj = nn.Linear(config.mlp_expansion*config.n_latent, config.n_latent)

    def forward(self,x):
        x = self.c_fc(x)
        x = self.act(x)
        x = self.c_proj(x)
        return x
    

class Block(nn.Module):
    def __init__(self,config):
        super().__init__()
        self.up_proj = nn.Linear(config.n_embed, config.n_latent)
        self.down_proj = nn.Linear(config.n_latent, config.n_embed)

        self.ln1 = nn.LayerNorm(config.n_latent,eps=1e-5,elementwise_affine=True)
        self.attn = GPT2Attention(config)
        self.ln2 = nn.LayerNorm(config.n_latent,eps=1e-5,elementwise_affine=True)
        self.mlp = GPT2MLP(config)

    def forward(self,x):

        h = self.up_proj(x)
        h = h + self.attn(self.ln1(h))
        h = h + self.mlp(self.ln2(h))
        
        return x + self.down_proj(h)


In [ ]:
class SinusoidalPositionEmbeddings(nn.Module):
    def __init__(self, dim):
        super().__init__()
        self.dim = dim

    def forward(self, time):
        device = time.device
        half_dim = self.dim // 2
        embeddings = math.log(10000) / (half_dim - 1)
        embeddings = torch.exp(torch.arange(half_dim, device=device) * -embeddings)
        embeddings = time[:, None] * embeddings[None, :]
        embeddings = torch.cat((embeddings.sin(), embeddings.cos()), dim=-1) #
        # TODO: Double check the ordering here
        return embeddings

In [ ]:
class LMEmbedding(nn.Module):
    def __init__(self,config):
        super().__init__()
        self.config = config

        self.embed = nn.Embedding(config.n_vocab,config.n_embed)
    
    def forward(self,input_ids):
        x = self.embed(input_ids)
        
        return x
        


In [ ]:
class Denoiser(nn.Module):
    def __init__(self,config):
        super().__init__()
        self.config = config

        self.transformer = nn.ModuleDict(dict(
            # wte = nn.Embedding(config.n_vocab,config.n_embed),
            wpe = nn.Embedding(config.n_context,config.n_embed),
            drop = nn.Dropout(0.1,inplace=False),
            h = nn.ModuleList(Block(config) for _ in range(config.n_layer)),
            ln_f = nn.LayerNorm(config.n_embed,eps=1e-5,elementwise_affine=True)
        ))
        
        # self.lm_head = nn.Linear(config.n_embed, config.n_vocab, bias=False)

        self.small_mlp = nn.Linear(config.n_embed, config.n_embed)

        self.time_embed = nn.Sequential(
            SinusoidalPositionEmbeddings(config.n_embed),
            nn.Linear(config.n_embed, config.n_embed),
            nn.GELU()
            )

    def forward(self,input_embeddings,time_step, targets=None):
        B,T,C = input_embeddings.size()
        device = input_embeddings.device

        pos = torch.arange(0,T,dtype=torch.long,device=device).unsqueeze(0)  # (1,T)
        x = input_embeddings +  self.transformer.wpe(pos)  # (B,T,C) pytorch does braodcasting for the position embeddingss and adds them to the token embeddings 
        
        time_emb = self.time_embed(time_step) # (B, C)
        x= x + time_emb.unsqueeze(1)  # (B, T, C)
        
        x = self.transformer.drop(x)


        for block in self.transformer.h:
            x = block(x)

        x = self.transformer.ln_f(x)  # (B,T,C)
        # logits = self.lm_head(x)  # (B,T,vocab_size) 
        # we don't need the head since we are not doing autoregressive language modeling
        
        # we want to predict the starting sequence before the noising part.
        x = self.small_mlp(x)  # (B,T,C)
        
        return x

In [ ]:
class Decoding(nn.Module):
    def __init__(self,config):
        super().__init__()
    # takes x0 (B,T,C) and give a softmax over vocab size           
        self.l1 = nn.Linear(config.n_embed, config.n_vocab, bias=False)
        
        
    def forward(self,x):
        x = self.l1(x)
        # x = F.softmax(x,dim=-1)

        return x

In [ ]:
class DiffusionLM(nn.Module):
    def __init__(self,config):
        super().__init__()
        self.config = config
        self.embedding = LMEmbedding(config)
        self.denoiser = Denoiser(config)
        self.decoder = Decoding(config)
        
    def forward(self,input_ids,time_step, targets=None):
        input_embeddings = self.embedding(input_ids)  # (B,T,C)
        x = self.denoiser(input_embeddings,time_step, targets)  # (B,T,C)
        logits = x@self.embedding.embed.weight.T  # (B,T,vocab_size)
        
        return x, logits

## Tokenizer

In [ ]:
class MyTokenizer:
    def __init__(self, max_len):
        tokenizer = tiktoken.get_encoding("r50k_base")
        self.special_tokens = {
            "<pad>": tokenizer.n_vocab,
            "<bos>": tokenizer.n_vocab + 1,
            "<eos>": tokenizer.n_vocab + 2,
        }
        print(tokenizer.n_vocab)
        self.tokenizer = tiktoken.Encoding(
            name="r50k_base_ext",
            pat_str=tokenizer._pat_str,
            mergeable_ranks=tokenizer._mergeable_ranks,
            special_tokens=self.special_tokens,
        )
        self.n_vocab = self.tokenizer.n_vocab 
        self.max_len = max_len

    @staticmethod
    def clean_text(text):
        for tok in ("<pad>", "<bos>", "<eos>"):
            text = text.replace(tok, "")
        return text
    
    def encode(self, text,max_len=None):
        if max_len is None:
            max_len = self.max_len
        # text = self.clean_text(text)
        ids = self.tokenizer.encode(text, allowed_special=set())
        ids = [self.special_tokens["<bos>"]] + ids + [self.special_tokens["<eos>"]]

        if len(ids) > max_len:
            ids = ids[:max_len]
            ids[-1] = self.special_tokens["<eos>"]
        else:
            ids += [self.special_tokens["<pad>"]] * (max_len - len(ids))
        return ids  

    def decode(self, ids):
        
        return self.tokenizer.decode(ids)


In [ ]:
tokenizer = MyTokenizer(max_len=13)
tokenizer.decode(tokenizer.encode("Hello, tiktoken is fast!"))

In [ ]:
config = gpt2config(n_vocab=tokenizer.n_vocab,n_embed=16,mlp_expansion=4,n_latent=512)
model = DiffusionLM(config).to(device)
print(f"Total Model parameters: {sum(p.numel() for p in model.parameters())/1e6:.2f}M")
print(config.n_vocab)

## Testing Initialization

In [ ]:
sample_input = "Once upon a time in a land far away, there lived a"
sample_tokens = tokenizer.encode(sample_input)
sample_input_ids = torch.tensor([sample_tokens], device=device)  # (1, sequence_length)
sample_time_step = torch.tensor([10], device=device)  # (1,)

In [ ]:
sample_input_ids.shape

In [ ]:
sample_output, sample_logits = model(sample_input_ids, sample_time_step)  # (1, sequence_length, n_embed)

def finalize_tokens(x0_final, embedding_weights):
    """
    Converts the final denoised latent into discrete token IDs.
    Args:
        x0_final: Tensor of shape (B, T, C)
        embedding_weights: Tensor of shape (Vocab, C)
    """
    # Fix: x2 must be 3D to match x1 (B, T, C)
    # Unsqueeze(0) makes it (1, Vocab, C), and PyTorch broadcasts it to (B, Vocab, C)
    distances = torch.cdist(x0_final, embedding_weights.unsqueeze(0), p=2) #(B,T,Vocab)  
    token_ids = torch.argmin(distances, dim=-1) #(B, T)
    
    return token_ids

token_ids = finalize_tokens(sample_output, model.embedding.embed.weight)
decoded_output = tokenizer.decode(token_ids.squeeze(0).tolist())
print("Decoded Text:",decoded_output)



## Forward Diffusion

In [ ]:
def fwd_diffusion(x0, t, alphas):
    """
    Directly samples x_t from x_0 at a specific timestep.
    
    Args:
        x0: Clean embeddings (B, SeqLen, EmbedDim) [cite: 126]
        t: Timesteps for the batch (B,) 
        alphas: Precomputed signal schedule from get_alphas()
    """
    # Select alpha_bar for each batch item and reshape for broadcasting
    a = alphas[t].view(-1, 1, 1).to(x0.device)
    
    # Sample Gaussian noise with same shape as x0
    noise = torch.randn_like(x0)
    
    # Formula: x_t = sqrt(alpha_bar) * x0 + sqrt(1 - alpha_bar) * noise [cite: 169]
    print("sqrt a avg:",torch.sqrt(a).mean())
    xt = torch.sqrt(a) * x0 + torch.sqrt(1 - a) * noise
    
    return xt

In [ ]:
# noisy_input = fwd_diffusion(model.embedding(sample_input_ids), torch.tensor([1000], device=device), alphas)

# token_ids = finalize_tokens(noisy_input, model.embedding.embed.weight)
# decoded_output = tokenizer.decode(token_ids.squeeze(0).tolist())
# print("Decoded Text:",decoded_output)

## Loading Datasets

In [ ]:
import pandas as pd

# Load E2E dataset - extract text from 'ref' column
df = pd.read_csv('datasets/e2e-dataset/trainset.csv')
text = ' '.join(df['ref'].tolist())

print(f"Dataset length: {len(text)} characters")
print(f"Number of samples: {len(df)}")
print(f"First sample: {df['ref'][0]}")

In [ ]:
# from datasets import load_dataset
# import pandas as pd

# # Load ROCStories dataset from HuggingFace
# rocstories = load_dataset("mintujupally/ROCStories")

# print(f"Dataset: {rocstories}")
# print(f"Train samples: {len(rocstories['train'])}")
# print(f"First story: {rocstories['train'][0]['text']}")

# # Convert to pandas DataFrame with 'ref' column
# train_df = pd.DataFrame({'ref': rocstories['train']['text']})
# test_df = pd.DataFrame({'ref': rocstories['test']['text']})

# # Save to CSV files
# train_df.to_csv('datasets/rocstories_train.csv', index=False)
# test_df.to_csv('datasets/rocstories_test.csv', index=False)

# print(f"\nSaved {len(train_df)} training samples to datasets/ROCStories/rocstories_train.csv")
# print(f"Saved {len(test_df)} test samples to datasets/ROCStories/rocstories_test.csv")

In [ ]:
# Split into train and test
train_size = int(0.9 * len(df))
train_df = df[:train_size].reset_index(drop=True)
test_df = df[train_size:].reset_index(drop=True)

print(f"Train samples: {len(train_df)}, Test samples: {len(test_df)}")

# Pre-encode all sequences for training efficiency
print("\nEncoding training data...")
train_encoded = []
for idx, row in train_df.iterrows():
    encoded = tokenizer.encode(row['ref'], max_len=64)  # Use fixed sequence length
    train_encoded.append(encoded)
    if (idx + 1) % 5000 == 0:
        print(f"Encoded {idx + 1}/{len(train_df)} train samples")

print("\nEncoding test data...")
test_encoded = []
for idx, row in test_df.iterrows():
    encoded = tokenizer.encode(row['ref'], max_len=64)
    test_encoded.append(encoded)

# Convert to tensors
train_encoded = torch.tensor(train_encoded, dtype=torch.long)
test_encoded = torch.tensor(test_encoded, dtype=torch.long)

print(f"\nTrain encoded shape: {train_encoded.shape}")
print(f"Test encoded shape: {test_encoded.shape}")

In [ ]:
# Data loader function - optimized to use pre-encoded data
def get_batch(split, batch_size=8, block_size=64, device=device):
    # Select the appropriate pre-encoded dataset
    data_encoded = train_encoded if split == 'train' else test_encoded
    
    # Randomly sample batch_size indices
    indices = torch.randint(0, len(data_encoded), (batch_size,))
    
    # Get the encoded sequences directly
    w_stack = data_encoded[indices].to(device)
    
    return w_stack

# Test batch
w_stack = get_batch('train', batch_size=4, block_size=64, device='cpu')
print(f"Batch shape: {w_stack.shape}")
print(f"First sequence decoded: {tokenizer.decode(w_stack[0].tolist())}")

## Training Loop


In [ ]:
# Training configuration
max_iters = 30000  
learning_rate = 3e-3
eval_iters = 1000  # Much fewer eval iterations (was 200!)
batch_size = 16  # Larger batch for better GPU utilization
sequence_length = 64
T = 500
num_timestep_samples = 8  # Sample 8 timesteps per iteration for better gradient estimate



In [ ]:
# Fixed alpha schedule - simple sqrt schedule
t = torch.arange(0, T+1, device=device, dtype=torch.float32)
alpha_bars = 1 - torch.sqrt(t / T)  # Goes from ~0 to 1-sqrt(1)=0
alpha_bars = torch.clamp(alpha_bars, min=0.001, max=0.999)
alphas = torch.zeros(T+1, device=device) #alpha_0 to alpha_T
alphas[0] = alpha_bars[0]
alphas[1:] = alpha_bars[1:] / alpha_bars[:-1]
alphas = torch.clamp(alphas, min=0.001, max=0.999)

# Precompute sqrt terms for efficiency
sqrt_ab = torch.sqrt(alpha_bars)
sqrt_1mab = torch.sqrt(1 - alpha_bars)

print(f"Alpha bars range: [{alpha_bars.min():.4f}, {alpha_bars.max():.4f}]")
print(f"Alphas range: [{alphas.min():.4f}, {alphas.max():.4f}]")


In [ ]:
model = torch.compile(model, mode='max-autotune')  # or 'max-autotune' for more optimization


In [ ]:
optimizer_model = torch.optim.AdamW(model.parameters(), lr=learning_rate,weight_decay=0.0)
lr_lambda = lambda step: 1.0 - (step / float(max_iters))
scheduler_model = LambdaLR(optimizer_model, lr_lambda=lr_lambda)

In [ ]:
def posterior_mean(x_t, x0, t, alpha_bars,alphas):
    abar_t = alpha_bars[t]
    abar_tm1 = alpha_bars[t-1] if t > 0 else torch.tensor(1.0, device=x_t.device)
    # print(abar_t,abar_tm1)
    coef1 = torch.sqrt(abar_tm1) * (1 - alphas[t]) / (1 - abar_t)
    coef2 = torch.sqrt(alphas[t]) * (1 - abar_tm1) / (1 - abar_t)
    # print("coef1 avg:",coef1.mean())
    # print("coef2 avg:",coef2.mean())
    return coef1 * x0 + coef2 * x_t

round_start = int(0.2 * max_iters)    # start after geometry is formed
round_warmup = int(0.5 * max_iters)   # ramp over 30% of training
round_max_weight = 0.4

def rounding_weight(it):
    # k controls steepness, x0 is the iteration where weight is 50%
    return 1
    k = 10 / round_warmup 
    x0 = round_start + (round_warmup / 2)
    weight = round_max_weight / (1 + math.exp(-k * (it - x0)))
    return weight if it >= round_start else 0.0


In [ ]:
for it in range(0,max_iters):

    w = get_batch('train', batch_size, sequence_length)   # (B,L)
    w_emb = model.embedding(w)                            # (B,L,d)

    # ---- sample x0 with small noise ----
    x0 = w_emb + 0.1 * torch.randn_like(w_emb)

    total_loss = 0.0

    
    # ---- Sample multiple timesteps to approximate E_t[loss] ----
    # This gives a better gradient estimate than sampling just one timestep
    eps = torch.randn_like(x0)
    denoising_loss = 0.0
    for _ in range(num_timestep_samples):
        t_random = torch.randint(1, T+1, (batch_size,), device=device)
        
        # Generate noised version at those timesteps
        t_idx = t_random   # Convert to 0-indexed
        sqrt_ab_t = sqrt_ab[t_idx].view(batch_size, 1, 1)
        sqrt_1mab_t = sqrt_1mab[t_idx].view(batch_size, 1, 1)
        
        xt = sqrt_ab_t * x0 + sqrt_1mab_t * eps
        x0_hat = model.denoiser(xt, t_random)
        
        # Accumulate denoising loss over sampled timesteps
        denoising_loss += F.mse_loss(x0_hat, x0)
    
    # Average over timestep samples
    # denoising_loss = denoising_loss / num_timestep_samples
    total_loss += denoising_loss
    
    # ---- Posterior mean regularization at timestep T ----
    # Sample at the final timestep T
    t_T = torch.full((batch_size,), T, device=device)
    xT = sqrt_ab[-1] * x0 + sqrt_1mab[-1] * eps
    
    # Predict x0 from xT
    x0_hat_T = model.denoiser(xT, t_T)
    
    # Compute posterior mean and regularize it to be close to zero
    # mu_hat_T = posterior_mean(xT, x0_hat_T, T, alpha_bars)
    mu_hat_T = posterior_mean(xT, x0, T, alpha_bars,alphas)

    posterior_loss = F.mse_loss(mu_hat_T, torch.zeros_like(mu_hat_T))
    total_loss += posterior_loss
    # posterior_loss = torch.tensor([0],device=device)
    # ---- t=1 anchor loss (every iteration for stability) ----
    # This ensures the denoised output matches actual word embeddings
    xt_1 = sqrt_ab[1] * x0 + sqrt_1mab[1] * torch.rand_like(x0)
    x0_hat_1 = model.denoiser(xt_1, torch.ones(batch_size, device=device))
    anchor_loss = F.mse_loss(x0_hat_1, w_emb)
    total_loss += anchor_loss

    # ---- rounding loss (tied weights) ----
    # Use x0_hat_1 from anchor step for rounding loss
    # logits = torch.matmul(x0_hat_1, model.embedding.embed.weight.T)  # (B,L,V)
    logits = x0_hat_1 @ model.embedding.embed.weight.T

    rounding_loss =  rounding_weight(it) * F.cross_entropy(logits.view(-1, config.n_vocab), w.view(-1))
    total_loss += rounding_loss  # Scale to balance losses
    # rounding_loss =torch.tensor([0],device=device)
    # ----/ optimize ----
    optimizer_model.zero_grad(set_to_none=True)
    total_loss.backward()
    
    # Gradient clipping to prevent exploding gradients
    # torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
    
    optimizer_model.step()
    scheduler_model.step()

    if it % eval_iters == 0:
        # print("loss dtypes:",type(total_loss),type(denoising_loss),type(posterior_loss),type(anchor_loss),type(rounding_loss))
        
        print(f"Iter {it}: loss = {total_loss.item():.4f}, denoising = {denoising_loss.item():.4f}, posterior = {posterior_loss.item():.4f}, anchor = {anchor_loss.item():.4f}, rounding = {rounding_loss.item():.4f}")

In [ ]:
with torch.no_grad():
    w = get_batch('train', 1, sequence_length)
    x0 = model.embedding(w)
    eps = torch.randn_like(x0)

    for t in [1, T//4, T//2, T]:
        xt = sqrt_ab[t-1] * x0 + sqrt_1mab[t-1] * eps
        print(t, torch.norm(xt - x0).item())


## Inference

In [ ]:
def reverse_diffusion_with_clamping(model, alpha_bars, T, context_length=50, batch_size=1, 
                                    clamping_start=0.4, skip_step=1, display_at_steps=None):
    """
    Reverse diffusion with selective progress display
    
    Args:
        display_at_steps: List of timesteps to display (e.g., [T, T//2, T//4, 1])
                         If None, only shows initial and final states
    """
    model.eval()
    
    # Start from pure noise: x_T ~ N(0, I)
    x_t = torch.randn(batch_size, context_length, config.n_embed, device=device)
    
    # Set default display steps if not provided
    if display_at_steps is None:
        display_at_steps = [1]  # Only final step by default
    
    print(f"\n{'='*70}")
    print(f"Starting Reverse Diffusion")
    print(f"{'='*70}")
    print(f"Total Timesteps: {T} | Context Length: {context_length}")
    print(f"Clamping Start: {clamping_start*100:.0f}% | Skip Step: {skip_step}")
    print(f"{'='*70}\n")
    
    # Show initial noisy state
    print(f"🌀 Initial State (t={T}, Pure Noise):")
    initial_tokens = finalize_tokens(x_t, model.embedding.embed.weight)
    initial_text = tokenizer.decode(initial_tokens[0].tolist())
    initial_text_clean = tokenizer.clean_text(initial_text)
    print(f"{initial_text_clean}")
    print(f"{'-'*70}\n")
    
    with torch.no_grad():
        for t_step in range(T, 0, -1):
            # Skip steps based on skip_step parameter
            if t_step % skip_step == 0 or t_step == T:
                pass
            else:
                continue
            
            t_tensor = torch.tensor([t_step] * batch_size, device=device)
            
            # Predict x0 from x_t
            x0_pred = model.denoiser(x_t, t_tensor)
            
            # Apply clamping in later timesteps
            if t_step > clamping_start * T:
                x0_clamped_tokens = finalize_tokens(x0_pred, model.embedding.embed.weight)
                x0_clamped = model.embedding(x0_clamped_tokens)
            else:
                x0_clamped = x0_pred
            
            # Sample noise for next step
            epsilon = torch.randn_like(x_t)
            
            # Compute x_{t-1}
            if t_step > 1:
                x_t = torch.sqrt(alpha_bars[t_step - 1]) * x0_clamped + \
                      torch.sqrt(1 - alpha_bars[t_step - 1]) * epsilon
            else:
                x_t = x0_clamped
            
            # Display only at specified timesteps (excluding initial which is already shown)
            if t_step in display_at_steps and t_step != T:
                generated_tokens = finalize_tokens(x0_clamped, model.embedding.embed.weight)
                generated_text = tokenizer.decode(generated_tokens[0].tolist())
                generated_text_clean = tokenizer.clean_text(generated_text)
                
                phase = "🔒 Clamping" if t_step > clamping_start * T else "✨ Refining"
                print(f"{phase} Intermediate State (t={t_step}):")
                print(f"{generated_text_clean}")
                # print(f"{'-'*70}\n")
    
    # Final output - always shown
    generated_tokens = finalize_tokens(x_t, model.embedding.embed.weight)
    generated_text = tokenizer.decode(generated_tokens[0].tolist())
    generated_text_clean = tokenizer.clean_text(generated_text)
    
    print(f"\n{'='*70}")
    print(f"\nFinal Output:")
    print(f"{generated_text_clean}")
    print(f"\n{'='*70}\n")
    
    return generated_tokens, generated_text_clean


In [ ]:
context_length = 64
generated_tokens, generated_text = reverse_diffusion_with_clamping(
    model=model,
    alpha_bars=alpha_bars,
    T=T,
    context_length=context_length,
    batch_size=1,
    clamping_start=0.8,
    skip_step=3,
    display_at_steps=[T//2,1]  # Show at 50% and final
)


## Visualizing the Embedding Space

In [ ]:
import matplotlib.pyplot as plt
# from mpl_toolkits.mplot3d import Axes3D
from sklearn.manifold import TSNE

def visualize_static_3d(emb_func, vocab_list, top_n=500):
    # 1. Force everything to CPU and Float32 immediately
    embeddings = emb_func.embed.weight[:top_n].detach().cpu().float().numpy()
    
    print(f"Running t-SNE on {top_n} points...")
    # Lower perplexity for fewer points to prevent hanging
    tsne = TSNE(n_components=3, perplexity=min(30, top_n-1), init='pca', verbose=1)
    embeds_3d = tsne.fit_transform(embeddings)

    # 2. Simple Matplotlib 3D plot
    fig = plt.figure(figsize=(10, 8))
    ax = fig.add_subplot(111, projection='3d')
    
    ax.scatter(embeds_3d[:, 0], embeds_3d[:, 1], embeds_3d[:, 2], alpha=0.6)
    
    # Label a few random points so you can see if words cluster
    for i in range(0, top_n, top_n // 10): 
        ax.text(embeds_3d[i, 0], embeds_3d[i, 1], embeds_3d[i, 2], vocab_list[i])

    plt.title("Static 3D Embedding View")
    plt.show()

In [ ]:
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
import numpy as np

@torch.no_grad()
def visualize_embeddings_2d(emb_func, vocab_list, top_n=5000):
    # 1. Extract and Clean Data
    # Convert to CPU, Float32, and Numpy immediately to prevent hanging
    embeddings = emb_func.weight[:top_n].detach().cpu().float().numpy()
    
    # 2. PCA Pre-reduction (768 -> 50)
    # This removes noise and makes t-SNE significantly faster and more stable
    print("Pre-reducing dimensions with PCA...")
    pca = PCA(n_components=8)
    embeddings_reduced = pca.fit_transform(embeddings)

    # 3. 2D t-SNE
    print(f"Running 2D t-SNE on {top_n} tokens...")
    # perplexity 30 is standard; init='pca' is faster than 'random'
    tsne = TSNE(n_components=2, perplexity=10, init='pca', verbose=1, random_state=42)
    embeds_2d = tsne.fit_transform(embeddings_reduced)

    # 4. POS Tagging using spaCy
    print("Performing POS tagging...")
    try:
        import spacy
        try:
            nlp = spacy.load("en_core_web_sm")
        except:
            print("Downloading spaCy model... (one-time setup)")
            import subprocess
            subprocess.run(["python", "-m", "spacy", "download", "en_core_web_sm"], check=True)
            nlp = spacy.load("en_core_web_sm")
        
        # Get POS tags for each token
        pos_tags = []
        for token_text in vocab_list[:top_n]:
            # Clean token text for spaCy processing
            clean_token = token_text.strip()
            if clean_token:
                doc = nlp(clean_token)
                pos = doc[0].pos_ if len(doc) > 0 else "OTHER"
            else:
                pos = "OTHER"
            pos_tags.append(pos)
    except Exception as e:
        print(f"Warning: Could not perform POS tagging ({e}), using default colors")
        pos_tags = ["OTHER"] * top_n
    
    # 5. Create color mapping for POS tags
    unique_pos = sorted(set(pos_tags))
    pos_to_color = {pos: i for i, pos in enumerate(unique_pos)}
    colors = [pos_to_color[pos] for pos in pos_tags]
    
    # Define a colormap with distinct colors
    cmap = plt.cm.get_cmap('tab20', len(unique_pos))
    
    # 6. Plotting with Matplotlib (Reliable & Fast)
    plt.figure(figsize=(14, 10))
    scatter = plt.scatter(embeds_2d[:, 0], embeds_2d[:, 1], 
                         alpha=0.6, s=8, c=colors, cmap=cmap)

    # Create legend
    handles = [plt.Line2D([0], [0], marker='o', color='w', 
                         markerfacecolor=cmap(pos_to_color[pos]), 
                         markersize=8, label=pos) 
              for pos in unique_pos]
    plt.legend(handles=handles, title="Part of Speech", 
              loc='center left', bbox_to_anchor=(1, 0.5), 
              frameon=True, fontsize=9)

    plt.title(f"Diffusion-LM Latent Space (Top {top_n} Tokens) - Colored by POS")
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()
    
    # Print POS distribution
    from collections import Counter
    pos_counts = Counter(pos_tags)
    print("\nPOS Tag Distribution:")
    for pos, count in pos_counts.most_common():
        print(f"  {pos:12s}: {count:5d} ({100*count/top_n:.1f}%)")


In [ ]:
my_vocab_itos_list = [tokenizer.decode([i]) for i in range(config.n_vocab-4)or range(config.n_vocab-3,config.n_vocab) ]

In [ ]:
visualize_embeddings_2d(model.embedding.embed, my_vocab_itos_list[:3000], top_n=3000)

## Saving/Load Model Checkpoints

In [ ]:
def save_checkpoint(model, config, alpha_bars, T, checkpoint_name, save_individual=True):
    
    checkpoint_dir = f'saved_models/checkpoints_{checkpoint_name}'
    os.makedirs(checkpoint_dir, exist_ok=True)
    
    # Prepare checkpoint dictionary
    checkpoint = {
        'config': config,
        'model_state_dict': model.state_dict(),
        'alpha_bars': alpha_bars,
        'T': T
    }
    
    # Save full checkpoint
    checkpoint_path = os.path.join(checkpoint_dir, 'diff_lm_checkpoint.pt')
    torch.save(checkpoint, checkpoint_path)
    print(f"✓ Checkpoint saved to {checkpoint_path}")
    
    # Optionally save individual model state dict
    if save_individual:
        model_path = os.path.join(checkpoint_dir, 'model.pt')
        torch.save(model.state_dict(), model_path)
        print(f"✓ Model state dict saved to {model_path}")
    
    print(f"\nCheckpoint summary:")
    print(f"  Directory: {checkpoint_dir}")
    print(f"  Model params: {sum(p.numel() for p in model.parameters())/1e6:.2f}M")
    print(f"  Timesteps (T): {T}")
    print(f"  Vocab size: {config.n_vocab}")


def load_checkpoint(checkpoint_name, device='cuda', eval_mode=True):

    checkpoint_dir = f'saved_models/checkpoints_{checkpoint_name}'
    checkpoint_path = os.path.join(checkpoint_dir, 'diff_lm_checkpoint.pt')
    
    # Load checkpoint
    print(f"Loading checkpoint from {checkpoint_path}...")
    checkpoint = torch.load(checkpoint_path, map_location=device)
    
    # Extract components
    loaded_config = checkpoint['config']
    loaded_alpha_bars = checkpoint['alpha_bars'].to(device)
    loaded_T = checkpoint['T']
    
    # Initialize model with loaded config
    model = DiffusionLM(loaded_config).to(device)
    
    # Handle compiled model state_dict (with _orig_mod. prefix)
    state_dict = checkpoint['model_state_dict']
    if any(key.startswith('_orig_mod.') for key in state_dict.keys()):
        print("Detected compiled model, removing '_orig_mod.' prefix...")
        state_dict = {key.replace('_orig_mod.', ''): value for key, value in state_dict.items()}
    
    model.load_state_dict(state_dict)
    
    if eval_mode:
        model.eval()
    
    # Precompute sqrt terms for inference
    sqrt_ab = torch.sqrt(loaded_alpha_bars)
    sqrt_1mab = torch.sqrt(1 - loaded_alpha_bars)
    
    print(f"\n✓ Model loaded successfully!")
    print(f"  Config: n_vocab={loaded_config.n_vocab}, n_layer={loaded_config.n_layer}, n_embed={loaded_config.n_embed}")
    print(f"  T={loaded_T}, Alpha bars range: [{loaded_alpha_bars.min():.4f}, {loaded_alpha_bars.max():.4f}]")
    print(f"  Total parameters: {sum(p.numel() for p in model.parameters())/1e6:.2f}M")
    
    return model, loaded_config, loaded_alpha_bars, loaded_T, sqrt_ab, sqrt_1mab


In [ ]:
# Save model checkpoint
save_checkpoint(
    model=model,
    config=config,
    alpha_bars=alpha_bars,
    T=T,
    checkpoint_name='1k_30k_E2E',  # Customize as needed
    save_individual=True
)


#### Loading Pre-trained Model

In [ ]:
# Load model checkpoint
model, config, alpha_bars, T, sqrt_ab, sqrt_1mab = load_checkpoint(
    checkpoint_name='1k_30k_E2E',  # Update as needed
    device=device,
    eval_mode=True
)


### Parts of Speech Controller 

In [ ]:
controller_config = gpt2config(n_vocab=tokenizer.n_vocab,n_layer=4)

In [ ]:
class POSClassifier(nn.Module):
    def __init__(self,config,pos_vocab):
        super().__init__()
        self.config = config
        self.embedding = LMEmbedding(config)
        self.denoiser = Denoiser(config)
        self.decoder = nn.Linear(config.n_embed, pos_vocab)  # Binary classification
        
    def forward(self,input_ids,time_step):
        input_embeddings = self.embedding(input_ids)  # (B,T,C)
        x = self.denoiser(input_embeddings,time_step)  # (B,T,C)
        logits = self.decoder(x)  # (B,T,pos_vocab)
        
        return logits

In [ ]:
controller = POSClassifier(controller_config,pos_vocab=10).to(device)